# Coffee Lovers Unite!

If caffeine is one of the most popular drugs, then coffee is likely one of the most popular delivery systems for it. Aside from caffeine, people enjoy the wonderful variety of coffee-related drinks. Let’s do a rough investigation of the “market share” by two of the top coffee chains in the United States!  

World Population Review provides some great data on store locations and chain prevalence. Check out this page for the Starbucks Coffee locations in the United States. Notice that this page only really gives the name of the state and the number of locations in that state.  
https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state  

# Scrape the Location Counts  
1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:  
- Starbucks  
- Dunkin’ Donuts  

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [24]:
# Starbucks Data

# Send a GET request to the webpage
response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")  # Parses the HTML content into a structured format
# Allows us to search through the HTML and extract specific elements

# Find the main table
table = soup.find("table", class_="wpr-table")

# Initialize an empty list to store data for rows
rows = []

# Iterate over each row in the table to collect store data by state, skipping the header row
for row in table.find_all("tr")[1:]:
    # Extract the state name from the <th> tag in each row
    state = row.find("th").get_text(strip=True)
    
    # Find all data cells in the row
    cells = row.find_all("td")

    # Get the text for each year's store count from the cells
    stores_2023 = cells[0].get_text(strip=True)
    stores_2021 = cells[1].get_text(strip=True)
    stores_2024 = cells[2].get_text(strip=True)
    
    # ChatGPT 4o Reccomedation: Append the extracted data for each state into the rows list as a dictionary
    rows.append({
        "state": state,
        "2023": stores_2023,
        "2021": stores_2021,
        "2024": stores_2024
    })

# Convert the list into a pandas DataFrame
starbucks = pd.DataFrame(rows)
starbucks.head()

,state,2023,2021,2024
0,California,"3,080","2,959","3,117"
1,Texas,"1,346","1,215","1,409"
2,Florida,844,786,892
3,Washington,741,739,736
4,New York,692,643,715


In [25]:
# Dunkin' Donuts Data

# Send a GET request to the webpage
response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")  # Parses the HTML content into a structured format
# Allows us to search through the HTML and extract specific elements

# Find the main table
table = soup.find("table", class_="wpr-table")

# Initialize an empty list to store data for rows
rows = []

# Iterate over each row in the table to collect store data by state, skipping the header row
for row in table.find_all("tr")[1:]:
    # Extract the state name from the <th> tag in each row
    state = row.find("th").get_text(strip=True)
    
    # Find all data cells in the row
    cells = row.find_all("td")

    # Get the text for each year's store count from the cells
    stores_2024 = cells[0].get_text(strip=True)
    stores_2023 = cells[1].get_text(strip=True)
    
    # Append the extracted data for each state into the rows list as a dictionary
    rows.append({
        "state": state,
        "2024": stores_2024,
        "2023": stores_2023
    })

# Convert the list into a pandas DataFrame
dunkin = pd.DataFrame(rows)
dunkin.head()

,state,2024,2023
0,New York,"1,431","1,414"
1,Massachusetts,"1,042","1,068"
2,Florida,909,883
3,New Jersey,872,866
4,Illinois,711,692


2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [36]:
# Convert Starbucks Data to long format
starbucks_long = starbucks.melt(
    id_vars=["state"],
    value_vars=["2023", "2021", "2024"],
    var_name="year",
    value_name="store_count"
)
starbucks_long["location"] = "Starbucks" # Add a new column to indicate the location
# starbucks_long.head()

# Convert Dunkin' Donuts Data to long format
dunkin_long = dunkin.melt(
    id_vars=["state"],
    value_vars=["2024", "2023"],
    var_name="year",
    value_name="store_count"
)
dunkin_long["location"] = "Dunkin"  # Add a new column to indicate the location
# dunkin_long.head()

# Merge the two datasets into one
coffee = pd.concat([starbucks_long, dunkin_long], ignore_index=True)
coffee.head()

,state,year,store_count,location
0,California,2023,"3,080",Starbucks
1,Texas,2023,"1,346",Starbucks
2,Florida,2023,844,Starbucks
3,Washington,2023,741,Starbucks
4,New York,2023,692,Starbucks


# Supplemental Data  
3. Scrape the state names and populations from this wikipedia page: https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population
Merge these data with your coffee dataset.

In [46]:
# Send a GET request to the webpage
response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")  # Parses the HTML content into a structured format

# Extract the table of interest
table = soup.find_all("table", class_ = "wikitable")[0] 

# Scrape population data from the table
rows = [] # Initialize an empty list to store data for rows

# Iterate over each row in the table, skipping the header row
for tr in table.find_all("tr")[1:]:
    cells = tr.find_all("td")
    
    # Ensure there are enough cells to avoid errors
    if len(cells) >= 4:
        # ChatGPT 4o Reccomedation: Get the state name, handling cases where it might be inside a nested <a> tag
        state_tag = cells[2].find("a") or cells[2]
        state = state_tag.get_text(strip=True)
        
        # Get the population value, handling cases with nested <a> tags
        pop_tag = cells[3].find("a") or cells[3]
        population = pop_tag.get_text(strip=True).replace(",", "")  # Remove commas from population
        
        # Append data to the list as a dictionary
        rows.append({
            "state": state,
            "population": int(population)  # Convert population to integer
        })

# Convert list to a pandas DataFrame
population = pd.DataFrame(rows)
# population.head()

# Merge the coffee dataset with the population dataset
coffee_pop = coffee.merge(population, on="state", how = "left")
coffee_pop.head()

# Convert number into integer: https://www.w3schools.com/python/ref_func_int.asp

,state,year,store_count,location,population
0,California,2023,"3,080",Starbucks,39538223
1,Texas,2023,"1,346",Starbucks,30145505
2,Florida,2023,844,Starbucks,21538187
3,Washington,2023,741,Starbucks,7705281
4,New York,2023,692,Starbucks,20201249


4. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

5. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

# Analyze
6. Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):  
- Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.  
- How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.  
- Does the distribution of each chain’s stores match population distribution, by both state/region?  
- Do the financial data match what you’d expect based on the number and locations of the stores? Why or why not?  

# Automate  
Convert your code for Exercises 1-3 above to a function that takes a single argument: the URL. This function should:  

- Scrape the information on state names and corresponding number of store locations on the webpage specified (assume the page has a table in the same form and placement as the ones you scraped above)  
- Extract the name of the company from either the URL specified or the webpage (assume the URL will have the same format as the ones used above)  
- Return a clean, organized and tidy dataset. Find a page other than Starbucks and Dunkin’ Donuts to test this on to confirm that it works. It’s fine if this is not related to coffee.  